In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from ld_script import *
pd.set_option('display.max_colwidth', None)

### First table is S0101 - Age and Sex

In [2]:
# path to the S0101 subject table
data_dir = 'data/subject_tables/unzipped_files/ACSST5Y2019.S0101_2021-01-12T120250'

In [3]:
# importing a file that contains the full fips codes for texas at tract level to ensure data integrity.
tract_checker = pd.read_csv('data/tx_tract_fips.csv')

In [4]:
S0101_data = pd.read_csv(os.path.join(data_dir, 'ACSST5Y2019.S0101_data_with_overlays_2021-01-08T174020.csv'), low_memory=False)
S0101_metadata = pd.read_csv(os.path.join(data_dir, 'ACSST5Y2019.S0101_metadata_2021-01-08T174020.csv'))

In [5]:
print(f'Shape of data table proper is {S0101_data.shape} \n'
    + f'Shape of metadata table is {S0101_metadata.shape}')

Shape of data table proper is (5266, 458) 
Shape of metadata table is (457, 2)


In [6]:
S0101_data.head(2)

,GEO_ID,NAME,S0101_C01_001E,S0101_C01_001M,S0101_C01_002E,S0101_C01_002M,S0101_C01_003E,S0101_C01_003M,S0101_C01_004E,S0101_C01_004M,...,S0101_C06_034E,S0101_C06_034M,S0101_C06_035E,S0101_C06_035M,S0101_C06_036E,S0101_C06_036M,S0101_C06_037E,S0101_C06_037M,S0101_C06_038E,S0101_C06_038M
0,id,Geographic Area Name,Estimate!!Total!!Total population,Margin of Error!!Total!!Total population,Estimate!!Total!!Total population!!AGE!!Under 5 years,Margin of Error!!Total!!Total population!!AGE!!Under 5 years,Estimate!!Total!!Total population!!AGE!!5 to 9 years,Margin of Error!!Total!!Total population!!AGE!!5 to 9 years,Estimate!!Total!!Total population!!AGE!!10 to 14 years,Margin of Error!!Total!!Total population!!AGE!!10 to 14 years,...,Estimate!!Percent Female!!Total population!!SUMMARY INDICATORS!!Age dependency ratio,Margin of Error!!Percent Female!!Total population!!SUMMARY INDICATORS!!Age dependency ratio,Estimate!!Percent Female!!Total population!!SUMMARY INDICATORS!!Old-age dependency ratio,Margin of Error!!Percent Female!!Total population!!SUMMARY INDICATORS!!Old-age dependency ratio,Estimate!!Percent Female!!Total population!!SUMMARY INDICATORS!!Child dependency ratio,Margin of Error!!Percent Female!!Total population!!SUMMARY INDICATORS!!Child dependency ratio,Estimate!!Percent Female!!Total population!!PERCENT ALLOCATED!!Sex,Margin of Error!!Percent Female!!Total population!!PERCENT ALLOCATED!!Sex,Estimate!!Percent Female!!Total population!!PERCENT ALLOCATED!!Age,Margin of Error!!Percent Female!!Total population!!PERCENT ALLOCATED!!Age
1,1400000US48001950100,"Census Tract 9501, Anderson County, Texas",4844,524,349,131,269,119,372,137,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)


In [7]:
S0101_metadata.head(2)

,GEO_ID,id
0,NAME,Geographic Area Name
1,S0101_C01_001E,Estimate!!Total!!Total population


We should break apart the GEOID field into its FIPS components for easier matching of datasets.

FIPS codes are 11 digits long. 2 state digits, 3 county digits, 6 tract digits.

In the GEO_ID column the FIPS code comes after the `US` characters.

In [9]:
# Custom function defined in ld_scripts.py in project folder to break apart GEO_ID column appropriately
S0101_data = parse_geo_id(S0101_data)

In [70]:
# 
S0101_data.FIPS.sort_values().equals(tract_checker.GEOID.astype(str).sort_values())

True

Note: We have created 4 additional columns using the GEO_ID column with indicies 458, 459, 460 and 461.

In [10]:
# Visual check to make sure these are properly formatted
S0101_data[['FIPS', 'STATEFP', 'COUNTYFP', 'TRACTCE']].head(5)

,FIPS,STATEFP,COUNTYFP,TRACTCE
0,fips,statefp,countyfp,tractce
1,48001950100,48,001,950100
2,48001950401,48,001,950401
3,48001950402,48,001,950402
4,48001950500,48,001,950500


The rows in the metadata table provide a mapping to the columns in the actual data table, but there are some minor offsets
to be made in order to make matching indices (i.e. row 1 in metadata table corresponds to column 1).

If we drop the GEO_ID and NAME columns in the dataset and drop the first row of the metadataset then they will be aligned.

In [11]:
S0101_data.drop(columns=['GEO_ID', 'NAME'], inplace=True)
S0101_metadata.drop(index=0, axis=0, inplace=True)
S0101_metadata.reset_index(drop=True, inplace=True)

# Subtracting 3 to S0101_data.shape[1] to account for the our creation of 3 new columns
assert (S0101_data.shape[1]-4 == S0101_metadata.shape[0]), print('mismatch in metadata and data correspondence')

In [12]:
int_locations = [3, *(list(range(15,30,2))), 43, 47, 61, 155, 307] # These are the rows as they appear in metadata excel sheet
int_locations = [x-3 for x in int_locations] # Offsetting these indicies to account for drops, header and 0-index pandas scheme

In [13]:
# Checking to make sure we have specified the columns we want.
S0101_metadata.iloc[int_locations]

,GEO_ID,id
0,S0101_C01_001E,Estimate!!Total!!Total population
12,S0101_C01_007E,Estimate!!Total!!Total population!!AGE!!25 to 29 years
14,S0101_C01_008E,Estimate!!Total!!Total population!!AGE!!30 to 34 years
16,S0101_C01_009E,Estimate!!Total!!Total population!!AGE!!35 to 39 years
18,S0101_C01_010E,Estimate!!Total!!Total population!!AGE!!40 to 44 years
20,S0101_C01_011E,Estimate!!Total!!Total population!!AGE!!45 to 49 years
22,S0101_C01_012E,Estimate!!Total!!Total population!!AGE!!50 to 54 years
24,S0101_C01_013E,Estimate!!Total!!Total population!!AGE!!55 to 59 years
26,S0101_C01_014E,Estimate!!Total!!Total population!!AGE!!60 to 64 years
40,S0101_C01_021E,Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!15 to 17 years


In [14]:
# Creating a list of the column names we want so we can use this to filter our data set.
wanted_columns = S0101_metadata.GEO_ID.iloc[int_locations].to_list()

In [ ]:
def create_wanted_columns(df1, df2, i_list):
    wc = df.GEO_ID.iloc[i_list].to_list()
    ac = list(df2).columns[-4:]
    wc.extend(ac)
    return wc
    

In [15]:
# Need to account for the 3 columns we created by breaking up GEO_ID that have no mapping to metdata df.
additional_columns = list(S0101_data.columns[-4:])
wanted_columns.extend(additional_columns)

In [16]:
wanted_columns

['S0101_C01_001E',
 'S0101_C01_007E',
 'S0101_C01_008E',
 'S0101_C01_009E',
 'S0101_C01_010E',
 'S0101_C01_011E',
 'S0101_C01_012E',
 'S0101_C01_013E',
 'S0101_C01_014E',
 'S0101_C01_021E',
 'S0101_C01_023E',
 'S0101_C01_030E',
 'S0101_C03_001E',
 'S0101_C05_001E',
 'FIPS',
 'STATEFP',
 'COUNTYFP',
 'TRACTCE']

In [17]:
S0101_data = S0101_data[wanted_columns]

In [18]:
S0101_data.head(3)

,S0101_C01_001E,S0101_C01_007E,S0101_C01_008E,S0101_C01_009E,S0101_C01_010E,S0101_C01_011E,S0101_C01_012E,S0101_C01_013E,S0101_C01_014E,S0101_C01_021E,S0101_C01_023E,S0101_C01_030E,S0101_C03_001E,S0101_C05_001E,FIPS,STATEFP,COUNTYFP,TRACTCE
0,Estimate!!Total!!Total population,Estimate!!Total!!Total population!!AGE!!25 to 29 years,Estimate!!Total!!Total population!!AGE!!30 to 34 years,Estimate!!Total!!Total population!!AGE!!35 to 39 years,Estimate!!Total!!Total population!!AGE!!40 to 44 years,Estimate!!Total!!Total population!!AGE!!45 to 49 years,Estimate!!Total!!Total population!!AGE!!50 to 54 years,Estimate!!Total!!Total population!!AGE!!55 to 59 years,Estimate!!Total!!Total population!!AGE!!60 to 64 years,Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!15 to 17 years,Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!18 to 24 years,Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!65 years and over,Estimate!!Male!!Total population,Estimate!!Female!!Total population,fips,statefp,countyfp,tractce
1,4844,252,197,335,163,312,447,318,318,250,205,1057,2486,2358,48001950100,48,001,950100
2,4838,843,907,753,444,419,314,223,73,0,634,84,4658,180,48001950401,48,001,950401


In [19]:
new_column_dictionary = dict(zip(S0101_data.columns, S0101_data.iloc[0]))

In [20]:
S0101_data.drop(index=0, axis=0, inplace=True)
S0101_data.reset_index(drop=True, inplace=True)

In [21]:
S0101_data.shape

(5265, 18)

In [22]:
S0101_data.head(3)

,S0101_C01_001E,S0101_C01_007E,S0101_C01_008E,S0101_C01_009E,S0101_C01_010E,S0101_C01_011E,S0101_C01_012E,S0101_C01_013E,S0101_C01_014E,S0101_C01_021E,S0101_C01_023E,S0101_C01_030E,S0101_C03_001E,S0101_C05_001E,FIPS,STATEFP,COUNTYFP,TRACTCE
0,4844,252,197,335,163,312,447,318,318,250,205,1057,2486,2358,48001950100,48,001,950100
1,4838,843,907,753,444,419,314,223,73,0,634,84,4658,180,48001950401,48,001,950401
2,7511,586,1339,1333,1171,980,790,485,315,5,268,214,7425,86,48001950402,48,001,950402


Reorder the columns and then compress the bins if necessary to align with our wanted columns.

In [23]:
curr_columns = list(S0101_data.columns)

One way we might be able to automate this is by using the last two entries in the list created by splitting with `!!`.
We can use the second to last entry as our keyword and then use the numeric characters in strings for ordering.

In [24]:
# Manual reordering might be necessary here
new_columns = ['S0101_C01_001E',
    'S0101_C01_021E',
    'S0101_C01_023E',
    'S0101_C01_007E',
    'S0101_C01_008E',
    'S0101_C01_009E',
    'S0101_C01_010E',
    'S0101_C01_011E',
    'S0101_C01_012E',
    'S0101_C01_013E',
    'S0101_C01_014E', 
    'S0101_C01_030E',
    'S0101_C03_001E',
    'S0101_C05_001E',
    'FIPS',
    'STATEFP',
    'COUNTYFP',
    'TRACTCE']

In [25]:
# Reorder columns using updated order
S0101_data = S0101_data[new_columns]

In [26]:
final_dict = {}
for i in new_columns:
    final_dict[i] = new_column_dictionary[i]

In [27]:
for i in final_dict.keys():
    if '!' in final_dict[i]:
        split_list = final_dict[i].split('!!')
        final_dict[i] = split_list[0] + ' ' + split_list[1] + ' ' + split_list[-1]

In [28]:
final_dict

{'S0101_C01_001E': 'Estimate Total Total population',
 'S0101_C01_021E': 'Estimate Total 15 to 17 years',
 'S0101_C01_023E': 'Estimate Total 18 to 24 years',
 'S0101_C01_007E': 'Estimate Total 25 to 29 years',
 'S0101_C01_008E': 'Estimate Total 30 to 34 years',
 'S0101_C01_009E': 'Estimate Total 35 to 39 years',
 'S0101_C01_010E': 'Estimate Total 40 to 44 years',
 'S0101_C01_011E': 'Estimate Total 45 to 49 years',
 'S0101_C01_012E': 'Estimate Total 50 to 54 years',
 'S0101_C01_013E': 'Estimate Total 55 to 59 years',
 'S0101_C01_014E': 'Estimate Total 60 to 64 years',
 'S0101_C01_030E': 'Estimate Total 65 years and over',
 'S0101_C03_001E': 'Estimate Male Total population',
 'S0101_C05_001E': 'Estimate Female Total population',
 'FIPS': 'fips',
 'STATEFP': 'statefp',
 'COUNTYFP': 'countyfp',
 'TRACTCE': 'tractce'}

In [29]:
S0101_data.columns

Index(['S0101_C01_001E', 'S0101_C01_021E', 'S0101_C01_023E', 'S0101_C01_007E',
       'S0101_C01_008E', 'S0101_C01_009E', 'S0101_C01_010E', 'S0101_C01_011E',
       'S0101_C01_012E', 'S0101_C01_013E', 'S0101_C01_014E', 'S0101_C01_030E',
       'S0101_C03_001E', 'S0101_C05_001E', 'FIPS', 'STATEFP', 'COUNTYFP',
       'TRACTCE'],
      dtype='object')

Now we need to decide which columns to merge together into age bins, since we know certain age groups 
do not need such a tight bucket and so we cut down on some features by collapsing columns together.

How to rename them then? Perhaps just modifying the very last part of column name.

In [30]:
S0101_data.head(3)

,S0101_C01_001E,S0101_C01_021E,S0101_C01_023E,S0101_C01_007E,S0101_C01_008E,S0101_C01_009E,S0101_C01_010E,S0101_C01_011E,S0101_C01_012E,S0101_C01_013E,S0101_C01_014E,S0101_C01_030E,S0101_C03_001E,S0101_C05_001E,FIPS,STATEFP,COUNTYFP,TRACTCE
0,4844,250,205,252,197,335,163,312,447,318,318,1057,2486,2358,48001950100,48,001,950100
1,4838,0,634,843,907,753,444,419,314,223,73,84,4658,180,48001950401,48,001,950401
2,7511,5,268,586,1339,1333,1171,980,790,485,315,214,7425,86,48001950402,48,001,950402


In [31]:
checker_series = S0101_data[['S0101_C01_007E','S0101_C01_008E']].copy()
# checker_series.sum(axis=1)

In [32]:
S0101_data[list(set(S0101_data.columns).difference(set(('FIPS', 'STATEFP', 'COUNTYFP', 'TRACTCE'))))] = \
S0101_data[list(set(S0101_data.columns).difference(set(('FIPS', 'STATEFP', 'COUNTYFP', 'TRACTCE'))))].apply(pd.to_numeric)

In [33]:
S0101_data.head()

,S0101_C01_001E,S0101_C01_021E,S0101_C01_023E,S0101_C01_007E,S0101_C01_008E,S0101_C01_009E,S0101_C01_010E,S0101_C01_011E,S0101_C01_012E,S0101_C01_013E,S0101_C01_014E,S0101_C01_030E,S0101_C03_001E,S0101_C05_001E,FIPS,STATEFP,COUNTYFP,TRACTCE
0,4844,250,205,252,197,335,163,312,447,318,318,1057,2486,2358,48001950100,48,001,950100
1,4838,0,634,843,907,753,444,419,314,223,73,84,4658,180,48001950401,48,001,950401
2,7511,5,268,586,1339,1333,1171,980,790,485,315,214,7425,86,48001950402,48,001,950402
3,4465,190,383,483,199,268,413,204,224,253,191,654,2273,2192,48001950500,48,001,950500
4,5148,344,448,552,355,484,204,182,334,279,373,730,2530,2618,48001950600,48,001,950600


In [34]:
S0101_data.shape

(5265, 18)

In [35]:
new, final_dict = merge_columns_by_add(S0101_data, 'S0101_C01_007E', ['S0101_C01_007E', 'S0101_C01_008E'], final_dict)

In [36]:
new, final_dict = merge_columns_by_add(new, 'S0101_C01_009E', ['S0101_C01_010E','S0101_C01_011E'], final_dict)

In [37]:
new, final_dict = merge_columns_by_add(new, 'S0101_C01_012E', ['S0101_C01_013E','S0101_C01_014E'], final_dict)

In [38]:
new.S0101_C01_007_8E.equals(checker_series.apply(pd.to_numeric).sum(axis=1))

True

In [39]:
display(new.shape)
new.head()

(5265, 13)

,S0101_C01_001E,S0101_C01_021E,S0101_C01_023E,S0101_C01_007_8E,S0101_C01_009_11E,S0101_C01_012_14E,S0101_C01_030E,S0101_C03_001E,S0101_C05_001E,FIPS,STATEFP,COUNTYFP,TRACTCE
0,4844,250,205,449,810,1083,1057,2486,2358,48001950100,48,001,950100
1,4838,0,634,1750,1616,610,84,4658,180,48001950401,48,001,950401
2,7511,5,268,1925,3484,1590,214,7425,86,48001950402,48,001,950402
3,4465,190,383,682,885,668,654,2273,2192,48001950500,48,001,950500
4,5148,344,448,907,870,986,730,2530,2618,48001950600,48,001,950600


In [40]:
new = new[~(new.S0101_C01_001E == 0)]

In [41]:
new.shape

(5228, 13)

In [42]:
S0101_md = pd.Series(final_dict)

In [43]:
S0101_md.sort_index(inplace=True)

In [44]:
pd_totals_path = 'data/processed_data/totals'
pd_percentages_path = 'data/processed_data/percentages'
pd_metadata_path = 'data/processed_data/metadata'

In [45]:
with open(os.path.join(pd_metadata_path, "S0101_medata.csv"), 'w', newline='') as outputfile:
    S0101_md.to_csv(outputfile, header=True, index=True)

In [46]:
with open(os.path.join(pd_totals_path, 'S0101_totals.csv'), 'w', newline='') as outputfile:
    new.to_csv(outputfile, header=True, index=False)    

In [47]:
pt_df = new.copy()

In [48]:
pt_df[list(set(new.columns).difference(set(('FIPS', 'STATEFP', 'COUNTYFP', 'TRACTCE'))))] = \
new[list(set(new.columns).difference(set(('FIPS', 'STATEFP', 'COUNTYFP', 'TRACTCE'))))].div(new.S0101_C01_001E, axis=0)

In [50]:
with open(os.path.join(pd_percentages_path, 'S0101_pct.csv'), 'w', newline='') as outputfile:
    pt_df.to_csv(outputfile, header=True, index=False)  

In [51]:
pt_df.rename(columns=final_dict)

,Estimate Total Total population,Estimate Total 15 to 17 years,Estimate Total 18 to 24 years,Estimate Total 25 to 34 years,Estimate Total 35 to 49 years,Estimate Total 50 to 64 years,Estimate Total 65 years and over,Estimate Male Total population,Estimate Female Total population,fips,statefp,countyfp,tractce
0,1.0,0.051610,0.042320,0.092692,0.167217,0.223576,0.218208,0.513212,0.486788,48001950100,48,001,950100
1,1.0,0.000000,0.131046,0.361720,0.334022,0.126085,0.017363,0.962795,0.037205,48001950401,48,001,950401
2,1.0,0.000666,0.035681,0.256291,0.463853,0.211690,0.028492,0.988550,0.011450,48001950402,48,001,950402
3,1.0,0.042553,0.085778,0.152744,0.198208,0.149608,0.146473,0.509071,0.490929,48001950500,48,001,950500
4,1.0,0.066822,0.087024,0.176185,0.168998,0.191531,0.141803,0.491453,0.508547,48001950600,48,001,950600
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5260,1.0,0.047601,0.098722,0.088535,0.196888,0.137989,0.116318,0.489906,0.510094,48505950400,48,505,950400
5261,1.0,0.015884,0.082873,0.199586,0.103591,0.178177,0.129144,0.466851,0.533149,48507950100,48,507,950100
5262,1.0,0.035448,0.078980,0.061567,0.179726,0.213930,0.160448,0.566542,0.433458,48507950200,48,507,950200
5263,1.0,0.045084,0.121343,0.116067,0.192806,0.162590,0.099760,0.603837,0.396163,48507950301,48,507,950301
